In [1]:
# import rasterio
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
# from shapely.geometry import Polygon, box
# import geopandas as gpd
import ee
import geemap
# from rasterio import features
import json
import os
import folium
# ee.Authenticate()
ee.Initialize()


In [2]:
from tools import filteringModule

In [3]:
help(ee.Filter)

Help on class Filter in module ee.filter:

class Filter(ee.computedobject.ComputedObject)
 |  Filter(*args, **kwargs)
 |  
 |  An object to represent collection filters.
 |  
 |  Method resolution order:
 |      Filter
 |      ee.computedobject.ComputedObject
 |      ee.encodable.Encodable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  Not(self)
 |      Returns the opposite of this filter.
 |      
 |      Returns:
 |        The negated filter, which will match iff this filter doesn't.
 |  
 |  __init__(self, opt_filter=None)
 |      Construct a filter.
 |      
 |      This constructor accepts the following args:
 |        1) Another filter.
 |        2) An array of filters (which are implicitly ANDed together).
 |        3) A ComputedObject returning a filter. Users shouldn't be making these;
 |           they're produced by the generator functions below.
 |      
 |      Args:
 |        opt_filter: Optional filter to add.
 |  
 |  predicateCount(self)
 |      Return

In [114]:
# folder = (new Function('return __dirname'))()
# filteringModule = require('filters.js')
city_name = 'Bangalore Urban'
admin2 = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')
selected = admin2.filter(ee.Filter.eq('ADM2_NAME', city_name))
geometry = selected.geometry()
geom = geometry.bounds()
# print('Bangalore Bounds:', geom)

year_start = '2020'
year_end = '2023'
image_before_start = '2015-11-01'
image_before_end = '2016-03-01'
image_after_start = '2023-01-01'
image_after_end = '2023-04-01'
DW_bands = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']
image_col_ = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1"). \
            filterDate(image_before_start, image_after_end) \
            .filterBounds(geom) \

mossaic = filteringModule.mosaicSamedayImage(image_col_)      
filled_col = filteringModule.forward_fill(mossaic)      
monthlyMedians = filteringModule.computeMonthlyMedians(filled_col)

# print(monthlyMedians)
# print(ee.Image(monthlyMedians.toList(monthlyMedians.size()).get(30)).bandNames().size())
monthlyMedians = monthlyMedians.map(filteringModule.addBandnumber).filter(ee.Filter.gt('numBands',0))

band = 'flooded_vegetation'
dw =  monthlyMedians.select(band)
beforeDw = dw.filterDate(image_before_start, image_before_end).mean()
afterDw = dw.filterDate(image_after_start, image_after_end).mean()
newUrban = beforeDw.lt(0.3).And(afterDw.gt(0.7))
lon, lat = geometry.centroid().getInfo()['coordinates']
# Map = folium.Map(location= [lat, lon], zoom_start=12)

Map = geemap.Map()
changeVisParams = {'min': 0, 'max': 1, 'palette': ['white', 'red']}
# Map.addLayer(dw.first().mask().clip(geometry), changeVisParams, 'Urban')
# Map.addLayer(dw.first().mask().Not(), {}, 'Urban')
Map.addLayer(newUrban, changeVisParams, "New Urban")
# Map.addLayer(monthlyMedians.first().select(['built', 'trees', 'water']).clip(geom), {}, 'Bangalore')
Map.centerObject(geometry, 12)
Map

Map(center=[12.942208526432214, 77.5868323785926], controls=(WidgetControl(options=['position', 'transparent_b…

In [84]:
# # Function to calculate the statistics for an image
# def calculateStatistics(image):
#     statistics = image.gt(0.7).reduceRegion(
#         reducer=ee.Reducer.sum(),  # Use the desired reducer function, such as mean, sum, max, etc.
#         geometry=geometry,
#         scale=20  # Set the appropriate scale based on the image resolution
#     )
#     return image.set(statistics)

# # Apply the calculateStatistics function over the image collection
# image_collection_with_stats = dw.map(calculateStatistics)

# # Get the resulting image collection with statistics
# image_collection_with_stats

In [85]:
# img1 = ee.Image(filled_col.toList(filled_col.size()).get(16))
# img2 = ee.Image(filled_col.toList(filled_col.size()).get(21))
# Map = geemap.Map()
# changeVisParams = {'min': 0, 'max': 1, 'palette': ['white', 'red']}
# # Map.addLayer(img1.clip(geometry))
# # Map.addLayer(dw.first().mask().Not(), {}, 'Urban')
# null_mask = img2.mask().Not()
# # Map.addLayer(null_mask)
# # Map.addLayer(img2.unmask(1).where(null_mask, img1).clip(geometry))
# Map.addLayer(img2)
# # Map.addLayer(monthlyMedians.first().select(['built', 'trees', 'water']).clip(geom), {}, 'Bangalore')
# Map.centerObject(geometry, 12)
# Map

In [87]:
reduced = dw.map(lambda image: ee.Feature(None, image.gt(0.7).reduceRegion(reducer = ee.Reducer.sum(), geometry= geom, scale = 20)\
                                          .set('month', image.get('month'))\
                                           .set('year', image.get('year')) ))
area_collection_dict = reduced.getInfo()

In [88]:
feature_list = area_collection_dict.get('features')
feature_count = len(feature_list)
areas = (np.array([i.get('properties').get(band) for i in feature_list])*20)**2
month_data = np.array([i.get('properties').get('month') for i in feature_list])
year_data = np.array([i.get('properties').get('year') for i in feature_list])
areas

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.60000000e+03, 1.60000000e+03, 1.60000000e+03, 4.00000000e+04,
       0.00000000e+00, 1.44400000e+05, 0.00000000e+00, 3.84400000e+05,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.50000000e+05, 1.44000000e+04,
       5.47600000e+05, 1.74240000e+06, 1.60000000e+03, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.04040000e+06, 4.00000000e+02, 5.76000000e+04,
       6.40000000e+03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.00000000e+06, 0.00000000e+00, 0.00000000e+00, 1.60000000e+03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.76400000e+05, 5.76000000e+06, 6.35040000e+06, 1.25440000e+06,
      

In [89]:
dir_path = f"app/data/{city_name}/{band}"
os.makedirs(dir_path, exist_ok=True)
with open(f"{dir_path}/areas.json", 'w') as dest:
    json.dump(feature_list, dest, indent = 6)

In [107]:
unique_year = np.sort(np.unique(year_data))[1:]
months_box = [True, True, True, True, True, True, False, False, False, False, False, False]
yearly_data = [np.mean(areas[year_data == i][0:6]) for i in unique_year]
yearly_data

[800.0,
 64066.666666666664,
 381933.3333333333,
 10666.666666666666,
 166933.33333333334,
 2288666.6666666665,
 100066.66666666667,
 168627866.66666666]

In [112]:
month_data[year_data == 2017]

array([ 1,  2,  3,  4,  5,  6,  7,  9, 11, 12])

array([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [91]:
import pandas as pd
import seaborn as sns

In [118]:
df = pd.DataFrame({"year": year_data, "month": month_data, "areas": areas})
df['date'] = pd.to_datetime(dict(year=df.year, month=df.month, day = 1))
df.set_index('date', inplace=True)
df = df.resample('MS').asfreq()
df.fillna(method='ffill', inplace=True)
df.month = df.index.month
df.year = df.index.year
df

,year,month,areas
date,,,
2015-12-01,2015,12,0.0
2016-01-01,2016,1,0.0
2016-02-01,2016,2,0.0
2016-04-01,2016,4,0.0
2016-05-01,2016,5,1600.0
...,...,...,...
2022-11-01,2022,11,55353600.0
2022-12-01,2022,12,11155600.0
2023-01-01,2023,1,119246400.0


In [93]:
import plotly.express as px

In [94]:
px.line(x=unique_year, y=yearly_data)

In [95]:
df_filtered = df[df.month.isin([1,2,3, 4])]
px.line(df, x='month', y='areas', color = 'year')